In [37]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain_huggingface import HuggingFaceEmbeddings


In [40]:
import os
from dotenv import load_dotenv

load_dotenv()  # reads from .env

api_key = os.getenv("PINECONE_API_KEY")
api_env = os.getenv("PINECONE_API_ENV")

print(api_key, api_env)


pcsk_2zM8sX_2MRCocUem7F35Da72ojPYJtyZX2uHvPwkpgwHcEQV5o2Dig4jsacQ46oxiWTbL4 gcp-starter


In [42]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader) 

    documents = loader.load()
    
    return documents

In [43]:
extracted_data = load_pdf("data/")

In [44]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [45]:
text_chunks = text_split(extracted_data)
print("length of my chunk: ", len(text_chunks))

length of my chunk:  5859


In [48]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()


In [49]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [50]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [52]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key=api_key)

index_name = "medicalchatbot"

if index_name not in [index["name"] for index in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=384, 
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)

vectorstore = PineconeVectorStore.from_texts(
    texts=[t.page_content for t in text_chunks],
    embedding=embeddings,
    index_name=index_name,
    pinecone_api_key=api_key
)


In [53]:
dosearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)
query = "What are Allergies"

docs = dosearch.similarity_search(query, k=2)
print("Result", docs)

Result [Document(id='7de942d1-5dbe-4bdd-82d5-6c2b1d342e62', metadata={}, page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of'), Document(id='6653b75e-f864-4915-83fe-cfa5d6554c01', metadata={}, page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reaction

In [54]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [55]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

In [61]:
llm = CTransformers(model= "model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens' : 512,
                            'temperature' : 0.8})

In [62]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = dosearch.as_retriever(search_kwargs={'k' : 2}),
    return_source_documents = True,
    chain_type_kwargs = chain_type_kwargs)

In [63]:
while True:
    user_input = input(f"Input Prompt:")
    result = qa({"query": user_input})
    print("Response:", result["result"])

/var/folders/c4/kbvyysv55vs2wm3lndrsfpsc0000gn/T/ipykernel_47900/1417358904.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": user_input})


Response: Acne is a common skin condition that occurs when the pores on the skin become clogged with oil, dead skin cells, and bacteria. It can cause redness, inflammation, and scarring on the face and other areas of the body.
